In [1]:
import pandas as pd
import pyodbc
import warnings
import google.generativeai as genai
import time
import random
import os
import json

In [ ]:
# Necesitaras crear un archivo GEMINI_API_KEYS.txt con las claves de Gemini que tengas
with open('../GEMINI_API_KEY.txt', 'r') as f:
    # Importante poner .strip() para quitar posibles '\n'
    genai.configure(api_key=f.readline().strip())

model = genai.GenerativeModel('gemini-2.0-flash')

# Conectarse a Azure SQL
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=sql-server-mamba.database.windows.net;'
    'DATABASE=sql-db-mamba;'
    'UID=sql-admin;'
    'PWD=serverpwd1!'
)

In [ ]:
PROMPT_GEMINI = """
Tengo una aplicación cuyo objetivo es describir especies en el contexto de rutas, de forma que el usuario puede personalizar las descripciones.
El usuario ha metido el siguiente texto:

"{prompt_usuario}"

Tu objetivo es personalizar la siguiente descripción realizando los mínimos cambios necesarios para que la nueva descripción se ajuste a lo que el usuario ha pedido.
Si el usuario te ha pedido algo que no es relevante para las descripciones o hace referencia a imágenes o a algo que no te he mencionado, deja la descripción como está.
La entrada te será introducida en tres bloques, "Descripción completa:::", "Descripción a adaptar:::" e "Información adicional:::".
Debes partir de la descripción a adaptar y generar una descripción adaptada, utilizando la información adicional y la descripción completa solamente si te son útiles.
Responde con únicamente la nueva descripción.
"""

EJEMPLO_ENTRADA = """
Descripción completa:::

{descripcion_defecto}

Descripción a adaptar:::

{descripcion_original}

Información adicional:::

{resultado_sql}
"""

In [ ]:
df_descripciones = pd.read_excel('../../2. Base de datos/DescripcionesEspecies.xlsx')
df_taxonomia = pd.read_excel('../../2. Base de datos/Taxonomia.xlsx')

In [7]:
train_dfs = []
test_dfs = []
for g in set(df_taxonomia['taxonomicgroup']):
    elems = df_taxonomia[(df_taxonomia['taxonomicgroup'] == g) & (df_taxonomia['nametype'] == 'Aceptado')]
    train_dfs.append(elems.sample(frac=0.9, random_state=42))
    test_dfs.append(elems[~elems['taxonid'].isin(train_dfs[-1]['taxonid'])])
train_df = pd.concat(train_dfs)[['taxonid', 'taxonomicgroup']]
test_df = pd.concat(test_dfs)[['taxonid', 'taxonomicgroup']]

In [8]:
train_df['taxonomicgroup'].value_counts()

taxonomicgroup
Plantas vasculares       667
Aves                     552
Invertebrados            519
Hongos                   425
Mamíferos                123
Reptiles                  77
Peces                     69
Anfibios                  30
Plantas no vasculares     23
Algas                      9
Cromistas y Bacterias      3
Name: count, dtype: int64

In [9]:
test_df['taxonomicgroup'].value_counts()

taxonomicgroup
Plantas vasculares       74
Aves                     61
Invertebrados            58
Hongos                   47
Mamíferos                14
Reptiles                  9
Peces                     8
Anfibios                  3
Plantas no vasculares     3
Algas                     1
Name: count, dtype: int64

In [ ]:
# ESTA CELDA SOLO SE DEBE CORRER UNA VEZ
warnings.filterwarnings('ignore')
df_prompts = pd.read_excel('PROMPTS.xlsx').drop(columns=['Texto usado', 'Tablas usadas', 'Columnas usadas'])
df_text2sql = pd.read_excel('Text2SQL_Data.xlsx')[['Prompt', 'SQL', 'Split']].drop_duplicates()
df_prompts['SQL'] = df_prompts['Prompt'].apply(lambda x: df_text2sql[df_text2sql['Prompt'] == x].iloc[0]['SQL'])
df_prompts['Split'] = df_prompts['Prompt'].apply(lambda x: df_text2sql[df_text2sql['Prompt'] == x].iloc[0]['Split'])
df_prompts['ResultadoSQL'] = ''
for i, row in df_prompts.iterrows():
    if row['ResultadoSQL'] == '':
        # Solo coger las 100 primeras filas --> reduce el tamaño de la consulta a unos limites aceptables
        query = row['SQL'].replace('<idtaxon>', str(row['idtaxon']))
        if 'SELECT DISTINCT' in query:  # TOP debe ir despues de DISTINCT
            query = query.replace('SELECT DISTINCT', 'SELECT DISTINCT TOP 100')
        else:
            query = query.replace('SELECT', 'SELECT TOP 100')
        df_sql_info = pd.read_sql(query, conn)
        df_prompts.loc[i, 'ResultadoSQL'] = json.dumps(df_sql_info.to_dict(orient='records'), indent=2)
warnings.filterwarnings('default')
df_prompts.to_excel('prompts_with_results.xlsx', index=False)
df_prompts

,idtaxon,Prompt,Nueva descripción,Prompt alternativo 1,Prompt alternativo 2,Descripción alternativa 1,Descripción alternativa 2,SQL,Split,ResultadoSQL
0,10729,Dime en qué otras provincias puedo encontrar e...,"Es una salamandra pequeña, de hasta 10 cm de l...","¿En qué lugares de España, a nivel provincial,...",Hazme una lista de provincias adicionales dond...,"Es una salamandra pequeña, de unos 10 centímet...","Es una salamandra pequeña, de hasta 10 centíme...",SELECT DISTINCT\n c.Provincia\nFROM\n Cu...,Train,"[\n {\n ""Provincia"": null\n },\n {\n ..."
1,1385,Dime en qué otras provincias puedo encontrar e...,"Es un árbol que puede crecer mucho, hasta 30 m...","¿En qué lugares de España, a nivel provincial,...",Hazme una lista de provincias adicionales dond...,"Es un árbol que puede ser muy alto, hasta 30 m...","Este árbol puede crecer mucho, normalmente has...",SELECT DISTINCT\n c.Provincia\nFROM\n Cu...,Train,"[\n {\n ""Provincia"": ""Las Palmas""\n },\n ..."
2,14085,Dime en qué otras provincias puedo encontrar e...,El fumarel cariblanco es un ave de tamaño simi...,"¿En qué lugares de España, a nivel provincial,...",Hazme una lista de provincias adicionales dond...,"El fumarel común se parece a los charranes, pe...",El fumarel común se distingue por su vuelo tra...,SELECT DISTINCT\n c.Provincia\nFROM\n Cu...,Train,"[\n {\n ""Provincia"": null\n },\n {\n ..."
3,10729,Me gustaría que me dijeras si estas especies a...,"Es una salamandra pequeña, de hasta 10 cm de l...",¿Puedes proporcionar datos sobre su presencia ...,Indica en qué otras comunidades autónomas se h...,"Es una salamandra pequeña, de unos 10 centímet...","Es una salamandra pequeña, de hasta 10 centíme...",SELECT DISTINCT\n c.CCAA\nFROM\n Cuadric...,Train,"[\n {\n ""CCAA"": null\n },\n {\n ""CCAA..."
4,1385,Me gustaría que me dijeras si estas especies a...,"Es un árbol que puede crecer mucho, hasta 30 m...",¿Puedes proporcionar datos sobre su presencia ...,Indica en qué otras comunidades autónomas se h...,"Es un árbol que puede ser muy alto, hasta 30 m...","Este árbol puede crecer mucho, normalmente has...",SELECT DISTINCT\n c.CCAA\nFROM\n Cuadric...,Train,"[\n {\n ""CCAA"": ""Canarias""\n },\n {\n ..."
...,...,...,...,...,...,...,...,...,...,...
121,24060,Rutas de menos de 15 km donde aparezcan,Este musgo forma tapices de color marrón oscur...,incluye ejemplos de rutas de menos de 15 km do...,en q rutas < 15 km se han registrado estas esp...,Este musgo forma extensiones de color marrón o...,Este musgo forma grupos de color marrón oscuro...,"SELECT\n r.Nombre_Ruta,\n r.Nombre_Etapa...",Train,"[\n {\n ""Nombre_Ruta"": ""Camino Natural de ..."
122,11595,Quiero la familia y otros nombres comunes,Familia - Felidae. Este felino se parece a un ...,Me interesa saber a qué familia pertenecen y q...,Puedes darme la familia y otros nombres comunes?,Familia - Felidae. Este felino se parece mucho...,Familia - Felidae. Este felino es parecido en ...,"SELECT\n t.family AS Familia,\n ne.nombr...",Train,[]
123,11378,Dame sinónimos y otros nombres comunes,Esta mariposa tiene colores diferentes según s...,"Añade otros nombres, tanto los comunes como lo...","¿Con qué otros nombres, científicos o comunes,...",Esta mariposa tiene colores distintos según si...,Esta mariposa es fácil de identificar por el c...,"SELECT\n ne.nombre_comun AS NombreComun,\n ...",Train,"[\n {\n ""NombreComun"": ""Sierra Nevada Blue..."
124,11985,En cuántas rutas salen y descripciones de una ...,"Rana de tamaño mediano (3–6 cm, hasta 8 cm), p...",Resume cada especie en una línea y añade en cu...,xq no me dices en cuants rutas se encuentra y ...,Rana mediana (3–6 cm) de piel lisa con granito...,"Rana adulta (3–6 cm, hasta 8 cm) con piel lisa...",SELECT\n COUNT(DISTINCT r.ID_Ruta) AS Numer...,Test,"[\n {\n ""NumeroDeRutas"": 167\n }\n]"


In [13]:
df_prompts = pd.read_excel('prompts_with_results.xlsx')

def build_messages(*, i, p1, p2, p3, index_id, elems, df_descripciones, df_info_str, index_desc, desc, descripciones):
    random.seed(i + index_id + index_desc) # Usar una semilla diferente
    prompt = random.sample([p1, p2, p3], k=1)
    # Crear few-shot-prompt
    messages = [
        {'role': 'user',
        'parts': [PROMPT_GEMINI.format(prompt_usuario=prompt)]},
        {'role': 'assistant',
        'parts': ['Entendido.']}
    ]
    species_k = random.sample(range(len(elems)), k=len(elems))
    for k in species_k:
        row = elems.iloc[k]
        example_initial_descs = df_descripciones[df_descripciones['idtaxon'] == row["idtaxon"]].iloc[0]
        description_k = random.sample([0, 1, 2], k=1)[0]
        if description_k % 3 == 0:
            example_final_desc = row["Nueva descripción"]
            example_initial_desc = example_initial_descs["Gemini1"]
        elif description_k % 3 == 1:
            example_final_desc = row["Descripción alternativa 1"]
            example_initial_desc = example_initial_descs["Gemini2"]
        else:
            example_final_desc = row["Descripción alternativa 2"]
            example_initial_desc = example_initial_descs["Gemini3"]
        messages.append(
            {'role': 'user',
            'parts': [EJEMPLO_ENTRADA.format(
                descripcion_defecto=example_initial_desc,
                descripcion_original=example_initial_descs['Descripcion'],
                resultado_sql=row['ResultadoSQL'])]}
        )
        messages.append(
            {'role': 'assistant',
            'parts': [example_final_desc]}
        )
    # Una vez añadidos los ejemplos, preguntar por la especie que realmente queremos
    messages.append(
        {'role': 'user',
        'parts': [EJEMPLO_ENTRADA.format(
            descripcion_defecto=desc,
            descripcion_original=descripciones['Descripcion'],
            resultado_sql=df_info_str)]},
    )
    return messages, example_initial_desc.strip()!=example_final_desc.strip()


def process_split(split, dataset_filename, split_df, elems_per_prompt):
    if not os.path.exists(dataset_filename):
        # Crear un JSON vacio si el archivo no existe
        with open(dataset_filename, 'w') as f:
            json.dump([], f)
    # Esto permite cargar datasets dejados a medias
    with open(dataset_filename, 'r') as file:
        outputs = json.load(file)
    warnings.filterwarnings('ignore')
    gemini_prompts = 0
    # Iterar sobre cada prompt
    prompts_set = set(df_prompts['Prompt'])
    for i, p1 in enumerate(prompts_set):
        if p1 in set([a['Prompt'] for a in outputs]):
            continue
        # Obtener prompts equivalentes
        elems = df_prompts[df_prompts['Prompt'] == p1]
        if elems.iloc[0]['Split'] != split:
            continue
        p2 = elems.iloc[0]['Prompt alternativo 1']
        p3 = elems.iloc[0]['Prompt alternativo 2']
        # Usamos un random_state diferente para cada prompt para coger elems_per_prompt especies diferentes sobre las que generar
        ids = split_df[~split_df['taxonid'].isin(elems['idtaxon'])]['taxonid'].sample(n=elems_per_prompt, random_state=i).values
        for index_id, id in enumerate(ids):
            print(f'Starting prompt {i+1} / {len(prompts_set)}, species {index_id+1} / {elems_per_prompt}...', end='\r')
            # Para cada especie, obtener sus descripciones
            descripciones = df_descripciones[df_descripciones['idtaxon'] == id].iloc[0]
            # Tambien obtener la informacion necesaria de la base de datos
            query = elems.iloc[0]['SQL'].replace("<idtaxon>", str(id))
            if 'SELECT DISTINCT' in query:  # TOP debe ir despues de DISTINCT
                query = query.replace('SELECT DISTINCT', 'SELECT DISTINCT TOP 100')
            else:
                query = query.replace('SELECT', 'SELECT TOP 100')
            df_sql_info = pd.read_sql(query, conn)
            df_info_str = json.dumps(df_sql_info.to_dict(orient='records'), indent=2)
            for index_desc, desc in enumerate([descripciones['Gemini1'], descripciones['Gemini2'], descripciones['Gemini3']]):
                messages, call_gemini = build_messages(
                    i=i, p1=p1, p2=p2, p3=p3, index_id=index_id, elems=elems, df_descripciones=df_descripciones,
                    df_info_str=df_info_str, index_desc=index_desc, desc=desc, descripciones=descripciones
                )
                if not call_gemini:
                    answer = desc
                else:
                    if gemini_prompts == 15:
                        time.sleep(60)
                        gemini_prompts = 0
                    gemini_prompts += 1
                    response = model.generate_content(messages)
                    try:
                        answer = response.candidates[0].content.parts[0].text.strip()
                    except:
                        print(f"Messages when exception: {messages}")
                        print(f"Response when exception: {response}")
                        answer = response.candidates[0].content.parts[0].text.strip()
                for p in [p1, p2, p3]:
                    outputs.append({
                        'idtaxon': int(id),
                        'Prompt': p,
                        'Descripcion': descripciones['Descripcion'],
                        'Gemini': desc,
                        'SQL': df_info_str,
                        'Nueva Descripcion': answer
                    })
        # Guardar el progreso para poder reanudar en caso de error o agotar la cuota diaria de llamadas a la API de Gemini
        with open(dataset_filename, 'w') as f:
            json.dump(outputs, f)
    warnings.filterwarnings('default')

In [ ]:
process_split('Train', '../description_train_dataset.json', train_df, 50)

In [ ]:
# Concatenar los ejemplos reales al dataset
with open('../description_train_dataset.json', 'r') as file:
    outputs = json.load(file)
for i, row in df_prompts[df_prompts['Split'] == 'Train'].iterrows():
    descripciones = df_descripciones[df_descripciones['idtaxon'] == row['idtaxon']].iloc[0]
    for p in [row['Prompt'], row['Prompt alternativo 1'], row['Prompt alternativo 2']]:
        for index_desc, desc in enumerate([descripciones['Gemini1'], descripciones['Gemini2'], descripciones['Gemini3']]):
            if index_desc == 0:
                answer = row['Nueva descripción']
            elif index_desc == 1:
                answer = row['Descripción alternativa 1']
            else:
                answer = row['Descripción alternativa 2']
            outputs.append({
                'idtaxon': row['idtaxon'],
                'Prompt': p,
                'Descripcion': descripciones['Descripcion'],
                'Gemini': desc,
                'SQL': row['ResultadoSQL'],
                'Nueva Descripcion': answer
            })
with open('../description_train_dataset.json', 'w') as f:
    json.dump(outputs, f)

In [ ]:
process_split('Test', '../description_test_dataset.json', test_df, 1)

In [ ]:
# Concatenar los ejemplos reales al dataset
with open('../description_test_dataset.json', 'r') as file:
    outputs = json.load(file)
for i, row in df_prompts[df_prompts['Split'] == 'Test'].iterrows():
    descripciones = df_descripciones[df_descripciones['idtaxon'] == row['idtaxon']].iloc[0]
    for p in [row['Prompt'], row['Prompt alternativo 1'], row['Prompt alternativo 2']]:
        for index_desc, desc in enumerate([descripciones['Gemini1'], descripciones['Gemini2'], descripciones['Gemini3']]):
            if index_desc == 0:
                answer = row['Nueva descripción']
            elif index_desc == 1:
                answer = row['Descripción alternativa 1']
            else:
                answer = row['Descripción alternativa 2']
            outputs.append({
                'idtaxon': row['idtaxon'],
                'Prompt': p,
                'Descripcion': descripciones['Descripcion'],
                'Gemini': desc,
                'SQL': row['ResultadoSQL'],
                'Nueva Descripcion': answer
            })
with open('../description_test_dataset.json', 'w') as f:
    json.dump(outputs, f)